In [1]:
import csv
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# path to parent directory of this script
#script_folder = Path(__file__).absolute().parent
# path to the file to read data from
file_location ='./object_data.csv'
# dataframes to store data
pre_post_time_df=pd.DataFrame(columns=['sim_id','end_msg', 'time_pre_hit', 'time_post_hit'])
box_properties_df=pd.DataFrame(columns=['sim_id','end_msg','size_x','size_y','size_z','com_x','com_y','com_z','mass','mu','mu2','izz'])
pre_hit_joints_df=pd.DataFrame(columns=['sim_id','end_msg','0','1','2','3','4','5','6'])
post_hit_joints_df=pd.DataFrame(columns=['sim_id','end_msg', '0','1','2','3','4','5','6'])
pre_hit_ee_df=pd.DataFrame(columns=['sim_id','end_msg', 'pos_x','pos_y','pos_z','quat_x','quat_y','quat_z','quat_w','roll','pitch','yaw','vel_x','vel_y','vel_z','ang_vel_x','ang_vel_y','ang_vel_z'])
post_hit_ee_df=pd.DataFrame(columns=['sim_id','end_msg', 'pos_x','pos_y','pos_z','quat_x','quat_y','quat_z','quat_w','roll','pitch','yaw','vel_x','vel_y','vel_z','ang_vel_x','ang_vel_y','ang_vel_z'])
pred_stop_pos_df=pd.DataFrame(columns=['sim_id','end_msg', 'time','pos_x','pos_y','pos_z'])
pre_hit_object_df=pd.DataFrame(columns=['sim_id','end_msg', 'time','pos_x','pos_y','pos_z','theta','vel_x','vel_y','vel_z','theta_dot'])
post_hit_object_df=pd.DataFrame(columns=['sim_id','end_msg', 'time','pos_x','pos_y','pos_z','theta','vel_x','vel_y','vel_z','theta_dot'])
post_hit_trajectory_df=pd.DataFrame(columns=['sim_id','end_msg', 'time','pos_x','pos_y','pos_z','theta','vel_x','vel_y','vel_z','theta_dot'])

In [3]:
def add_line(df,line,sim_id):
    line=[float(x) for x in line]
    line.insert(0,sim_id) # sim_id column
    line.insert(1,-1) # end_msg column initialized with -1
    new_df_line=pd.DataFrame([line],columns=df.columns)
    return pd.concat([df,new_df_line],ignore_index=True)

def set_end_msg(sim_id,end_type):
    '''
    this function specify the type of end message that the simulation stopped with in the column end_msg
    '''
    pre_post_time_df.loc[pre_post_time_df['sim_id']==sim_id, ['end_msg']] = end_type
    box_properties_df.loc[box_properties_df['sim_id']==sim_id, ['end_msg']] = end_type
    pre_hit_joints_df.loc[pre_hit_joints_df['sim_id']==sim_id, ['end_msg']] = end_type
    post_hit_joints_df.loc[post_hit_joints_df['sim_id']==sim_id, ['end_msg']] = end_type
    pre_hit_ee_df.loc[pre_hit_ee_df['sim_id']==sim_id, ['end_msg']] = end_type
    post_hit_ee_df.loc[post_hit_ee_df['sim_id']==sim_id, ['end_msg']] = end_type
    pred_stop_pos_df.loc[pred_stop_pos_df['sim_id']==sim_id, ['end_msg']] = end_type
    pre_hit_object_df.loc[pre_hit_object_df['sim_id']==sim_id, ['end_msg']] = end_type
    post_hit_object_df.loc[post_hit_object_df['sim_id']==sim_id, ['end_msg']] = end_type
    post_hit_trajectory_df.loc[post_hit_trajectory_df['sim_id']==sim_id, ['end_msg']] = end_type
    
def delete_sim(sim_ids):
    '''
    this function deletes the dataframes' rows of the simulation with the ids in parameter 
    '''
    pre_post_time_df.drop(pre_post_time_df[pre_post_time_df.sim_id.isin(sim_ids)].index, inplace=True)
    box_properties_df.drop(box_properties_df[box_properties_df.sim_id.isin(sim_ids)].index, inplace=True)
    pre_hit_joints_df.drop(pre_hit_joints_df[pre_hit_joints_df.sim_id.isin(sim_ids)].index, inplace=True)
    post_hit_joints_df.drop(post_hit_joints_df[post_hit_joints_df.sim_id.isin(sim_ids)].index, inplace=True)
    pre_hit_ee_df.drop(pre_hit_ee_df[pre_hit_ee_df.sim_id.isin(sim_ids)].index, inplace=True)
    post_hit_ee_df.drop(post_hit_ee_df[post_hit_ee_df.sim_id.isin(sim_ids)].index, inplace=True)
    pred_stop_pos_df.drop(pred_stop_pos_df[pred_stop_pos_df.sim_id.isin(sim_ids)].index, inplace=True)
    pre_hit_object_df.drop(pre_hit_object_df[pre_hit_object_df.sim_id.isin(sim_ids)].index, inplace=True)
    post_hit_object_df.drop(post_hit_object_df[post_hit_object_df.sim_id.isin(sim_ids)].index, inplace=True)
    post_hit_trajectory_df.drop(post_hit_trajectory_df[post_hit_trajectory_df.sim_id.isin(sim_ids)].index, inplace=True)

df_choices = {'pre_post_time': pre_post_time_df,
           'box_properties': box_properties_df,
           'pre_hit_joints': pre_hit_joints_df,
           'post_hit_joints': post_hit_joints_df,
           'pre_hit_ee': pre_hit_ee_df,
           'post_hit_ee': post_hit_ee_df,
           'pred_stop_pos': pred_stop_pos_df,
           'pre_hit_object': pre_hit_object_df,
           'post_hit_object': post_hit_object_df,
           'post_hit_trajectory:': post_hit_trajectory_df,
           'end': 'end',
           }
end_choices = {' free v=0':0,
               ' free m3':1,
               ' reset':2,
               ' stopped':3,   
            }
bug=False
bug_num=0
error_num=0
track=False
sim_id=0
first_end=True
end_type=-1
post_empty_line=False

In [4]:
with open(file_location, 'r') as f:          # read lines separately
    reader = csv.reader(f, delimiter=',')
    for i, line in enumerate(reader):
        if line : # if line not empty
            #print(i,line)
            if line[0] == 'end':
                track=False
                if first_end:
                    first_end=False
                    continue
                elif post_empty_line:
                    #delete the last simulation of each run because it's incomplete
                    delete_sim([sim_id-1])
                    post_empty_line=False
                    continue
                elif bug:
                    bug=False
                    post_empty_line=False
                    continue
                else:
                    end_type = end_choices.get(line[1])
                    if end_type is None : error_num+=1
                    set_end_msg(sim_id,end_type)
                    #print(i,'end of sim ',sim_id,'with',line[1],' ',end_type)
                    sim_id+=1
            elif line[0] == 'post_hit_trajectory:':
                #print(i,' tracking will start')
                track=True
                post_empty_line=False
                continue
            elif track:
                post_hit_trajectory_df=add_line(post_hit_trajectory_df,line,sim_id)
            elif (track==False) and (df_choices.get(line[0]) is None):
                print(i+1,' bug')
                bug_num+=1
                bug=True
                #after bug there is a line that begins with end
            else:
                if(line[0]=='pre_post_time') : pre_post_time_df=add_line(pre_post_time_df,line[1:],sim_id)
                elif(line[0]=='box_properties') : box_properties_df=add_line(box_properties_df,line[1:],sim_id)
                elif(line[0]=='pre_hit_joints') : pre_hit_joints_df=add_line(pre_hit_joints_df,line[1:],sim_id)
                elif(line[0]=='post_hit_joints') : post_hit_joints_df=add_line(post_hit_joints_df,line[1:],sim_id)
                elif(line[0]=='pre_hit_ee') : pre_hit_ee_df=add_line(pre_hit_ee_df,line[1:],sim_id)
                elif(line[0]=='post_hit_ee') : post_hit_ee_df=add_line(post_hit_ee_df,line[1:],sim_id)
                elif(line[0]=='pred_stop_pos') : pred_stop_pos_df=add_line(pred_stop_pos_df,line[1:],sim_id)
                elif(line[0]=='pre_hit_object') : pre_hit_object_df=add_line(pre_hit_object_df,line[1:],sim_id)
                elif(line[0]=='post_hit_object') : post_hit_object_df=add_line(post_hit_object_df,line[1:],sim_id)
                #print(data)
            post_empty_line=False               
        else:
            post_empty_line=True
            #print(i," empty line")

1448  bug
9940  bug
10223  bug
10830  bug
11681  bug
28687  bug
29060  bug
29418  bug
29893  bug
31043  bug
32093  bug
33321  bug
33898  bug
34399  bug
35733  bug
39098  bug
39833  bug
39873  bug
40437  bug
51189  bug
52063  bug
58954  bug


In [5]:
print('end of data collection! number of simulations read :',sim_id)
print(bug_num,' bugs, ', error_num,' errors')
print(pre_post_time_df.shape,box_properties_df.shape,pre_hit_joints_df.shape,post_hit_joints_df.shape,
      pre_hit_ee_df.shape,post_hit_ee_df.shape,pred_stop_pos_df.shape,pre_hit_object_df.shape,post_hit_object_df.shape,post_hit_trajectory_df.shape  )


end of data collection! number of simulations read : 2567
22  bugs,  0  errors
(2546, 4) (2546, 12) (2546, 9) (2546, 9) (2546, 18) (2546, 18) (2546, 6) (2546, 11) (2546, 11) (27914, 11)


In [6]:
nb_0=len(post_hit_object_df[post_hit_object_df.end_msg==0]) #v=0
nb_1=len(post_hit_object_df[post_hit_object_df.end_msg==1]) #hit mode
nb_2=len(post_hit_object_df[post_hit_object_df.end_msg==2]) #reset
nb_3=len(post_hit_object_df[post_hit_object_df.end_msg==3]) #stopped
total=len(post_hit_object_df)

print('simulations ended with v0',nb_0)
print('simulations ended with hit mode',nb_1)
print('simulations ended with stopped',nb_3)
print('simulations ended with reset',nb_2)
print('total number of simulations: ',total)
print('total usefull simulations: ',total-nb_2-nb_3)

simulations ended with v0 1
simulations ended with hit mode 2496
simulations ended with stopped 32
simulations ended with reset 17
total number of simulations:  2546
total usefull simulations:  2497


In [7]:
# drop the simulations that end with 'stopped' or reset 
ids_to_drop=list(post_hit_object_df[~post_hit_object_df.end_msg.isin([0, 1])].sim_id)
delete_sim(ids_to_drop)
post_hit_object_df

,sim_id,end_msg,time,pos_x,pos_y,pos_z,theta,vel_x,vel_y,vel_z,theta_dot
0,0,1,3.912,0.549746,-0.392283,0.165000,0.001144,-0.016881,0.531760,0.003855,0.112736
1,1,1,5.572,0.544664,0.153506,0.165000,0.000870,0.012707,-0.573172,0.005198,0.128608
2,2,1,7.162,0.541010,-0.341818,0.164996,0.003741,-0.035960,0.591660,-0.003860,0.257084
3,3,1,8.802,0.545044,0.197883,0.165000,0.002211,0.028519,-0.568039,0.004609,0.258527
4,4,1,10.422,0.544487,-0.325656,0.165000,0.003707,-0.035582,0.590104,0.003746,0.529514
...,...,...,...,...,...,...,...,...,...,...,...
2541,2562,1,537.563,0.544176,0.271015,0.164997,0.030599,-0.012734,-0.510231,-0.002801,-0.931129
2542,2563,1,539.203,0.541301,-0.269336,0.165000,-0.008840,-0.023821,0.574031,0.003676,0.852427
2543,2564,1,540.833,0.538322,0.265571,0.164996,0.008942,-0.029059,-0.588666,-0.003738,-0.204051
2544,2565,1,542.463,0.538532,-0.264310,0.164996,-0.007857,0.039315,0.610824,-0.003597,-0.312522


In [8]:
first_trajectory_pos=post_hit_object_df[post_hit_object_df.sim_id.diff(periods=-1)==-1].iloc[:, 2:].reset_index(drop=True)
first_trajectory_pos

,time,pos_x,pos_y,pos_z,theta,vel_x,vel_y,vel_z,theta_dot
0,3.912,0.549746,-0.392283,0.165000,0.001144,-0.016881,0.531760,0.003855,0.112736
1,5.572,0.544664,0.153506,0.165000,0.000870,0.012707,-0.573172,0.005198,0.128608
2,7.162,0.541010,-0.341818,0.164996,0.003741,-0.035960,0.591660,-0.003860,0.257084
3,8.802,0.545044,0.197883,0.165000,0.002211,0.028519,-0.568039,0.004609,0.258527
4,10.422,0.544487,-0.325656,0.165000,0.003707,-0.035582,0.590104,0.003746,0.529514
...,...,...,...,...,...,...,...,...,...
2445,535.933,0.548162,-0.278106,0.165000,-0.012105,-0.024882,0.531548,0.003520,0.927565
2446,537.563,0.544176,0.271015,0.164997,0.030599,-0.012734,-0.510231,-0.002801,-0.931129
2447,539.203,0.541301,-0.269336,0.165000,-0.008840,-0.023821,0.574031,0.003676,0.852427
2448,540.833,0.538322,0.265571,0.164996,0.008942,-0.029059,-0.588666,-0.003738,-0.204051


In [9]:
# Only take the simulations that have a corresponding first pose
end_pos=pre_hit_object_df[pre_hit_object_df.sim_id.diff(periods=1)==1].iloc[:, 2:].reset_index(drop=True)
end_pos

,time,pos_x,pos_y,pos_z,theta,vel_x,vel_y,vel_z,theta_dot
0,5.542,0.544362,0.164311,0.164994,-0.002345,-2.132440e-03,0.000290,-0.005545,-7.422800e-08
1,7.132,0.542186,-0.360492,0.164997,-0.009781,4.954050e-07,-0.000254,-0.000059,3.948910e-04
2,8.772,0.544371,0.208369,0.164996,-0.003110,3.182660e-03,-0.000002,0.000252,1.754260e-04
3,10.392,0.545381,-0.340595,0.164997,-0.013830,1.763940e-06,-0.003682,-0.000335,5.481570e-04
4,12.022,0.542682,0.222710,0.164996,0.021115,-1.048020e-05,0.006482,-0.000157,-1.041880e-03
...,...,...,...,...,...,...,...,...,...
2445,537.533,0.544322,0.276469,0.164992,0.041264,1.178090e-04,0.003561,-0.003159,-2.751680e-03
2446,539.173,0.541819,-0.281863,0.165000,-0.030011,1.118310e-04,-0.002451,0.004339,3.349360e-08
2447,540.803,0.539106,0.280674,0.164995,0.017387,1.915960e-05,0.003018,-0.000558,-9.599360e-04
2448,542.433,0.537559,-0.278416,0.164996,0.001780,7.246270e-06,-0.003583,-0.004050,-1.249280e-07


# End effector 1

In [22]:
initial_pos = first_trajectory_pos[(end_pos.pos_y>0) & (abs(end_pos.vel_y)<0.005)].reset_index(drop=True)
final_pos = end_pos[(end_pos.pos_y>0) & (abs(end_pos.vel_y)<0.005)].reset_index(drop=True)
initial_pos

,time,pos_x,pos_y,pos_z,theta,vel_x,vel_y,vel_z,theta_dot
0,3.912,0.549746,-0.392283,0.165000,0.001144,-0.016881,0.531760,0.003855,0.112736
1,7.162,0.541010,-0.341818,0.164996,0.003741,-0.035960,0.591660,-0.003860,0.257084
2,13.682,0.539453,-0.319696,0.164997,-0.006127,-0.025347,0.573208,-0.003187,0.742820
3,16.932,0.542481,-0.313468,0.164999,0.008898,0.028678,0.554703,-0.001395,-0.302035
4,20.202,0.546113,-0.289720,0.165000,0.015767,-0.039648,0.594558,0.003626,0.512931
...,...,...,...,...,...,...,...,...,...
1128,526.163,0.538912,-0.279268,0.164997,-0.012413,-0.028927,0.524144,-0.002766,0.549641
1129,529.413,0.541216,-0.270930,0.164996,0.002787,0.023405,0.569600,-0.003929,-0.762786
1130,532.673,0.549267,-0.280814,0.164997,0.024337,0.014553,0.548063,-0.003171,-0.637602
1131,535.933,0.548162,-0.278106,0.165000,-0.012105,-0.024882,0.531548,0.003520,0.927565


In [23]:
final_pos

,time,pos_x,pos_y,pos_z,theta,vel_x,vel_y,vel_z,theta_dot
0,5.542,0.544362,0.164311,0.164994,-0.002345,-2.132440e-03,0.000290,-0.005545,-7.422800e-08
1,8.772,0.544371,0.208369,0.164996,-0.003110,3.182660e-03,-0.000002,0.000252,1.754260e-04
2,15.282,0.538041,0.231240,0.164995,0.011297,-1.336190e-05,0.003959,-0.001085,-5.794640e-04
3,18.552,0.544492,0.244797,0.164997,-0.005699,-1.599200e-06,0.003794,0.000579,2.463050e-04
4,21.802,0.544723,0.249887,0.164993,0.025621,5.878480e-05,0.001640,-0.007201,-1.062790e-08
...,...,...,...,...,...,...,...,...,...
1128,527.763,0.537365,0.275712,0.164997,0.003295,9.517750e-07,0.003914,0.000556,-1.420150e-04
1129,531.023,0.545651,0.271432,0.164997,-0.035225,2.297990e-03,-0.000080,-0.002599,1.409570e-08
1130,534.273,0.551480,0.274025,0.164997,-0.014986,1.629180e-06,0.002737,0.000083,6.145860e-04
1131,537.533,0.544322,0.276469,0.164992,0.041264,1.178090e-04,0.003561,-0.003159,-2.751680e-03


In [24]:
# tensors
x0_ = torch.tensor(initial_pos.pos_y)
xf_ = torch.tensor(final_pos.pos_y)
dt_ = torch.tensor(final_pos.time - initial_pos.time )
v0_ = torch.tensor(initial_pos.vel_y)
mu_ = torch.full(x0_.size(), 0.1, requires_grad=True)

In [25]:
# gravity constant
g = 9.81
#loss
loss = nn.MSELoss()
cost = 1e+8
#optimizer
optimizer = optim.SGD([mu_], lr=0.01, momentum=0.9)

In [26]:
while(cost)>1e-8:
    optimizer.zero_grad()
    xf_est_ = x0_+v0_*dt_-0.5*mu_*g*dt_**2
    cost = loss(xf_est_,xf_)
    cost.backward()
    optimizer.step()

In [27]:
mu_.mean()

tensor(0.0287, grad_fn=<MeanBackward0>)

# End effector 2

In [28]:
initial_pos = first_trajectory_pos[(end_pos.pos_y<0) & (abs(end_pos.vel_y)<0.005)].reset_index(drop=True)
final_pos = end_pos[(end_pos.pos_y<0) & (abs(end_pos.vel_y)<0.005)].reset_index(drop=True)
initial_pos

,time,pos_x,pos_y,pos_z,theta,vel_x,vel_y,vel_z,theta_dot
0,5.572,0.544664,0.153506,0.165000,0.000870,0.012707,-0.573172,0.005198,0.128608
1,8.802,0.545044,0.197883,0.165000,0.002211,0.028519,-0.568039,0.004609,0.258527
2,12.052,0.542551,0.219992,0.165016,0.016656,-0.022533,-0.522189,-0.006103,-0.782916
3,15.312,0.538033,0.217971,0.164996,0.011308,-0.000021,-0.580862,0.001084,-0.000483
4,18.582,0.545307,0.231834,0.164996,0.005550,0.036524,-0.587976,-0.003529,0.357873
...,...,...,...,...,...,...,...,...,...
1103,527.793,0.538512,0.259992,0.164997,0.014797,0.039163,-0.601216,-0.002924,0.303389
1104,531.053,0.545975,0.262424,0.165000,-0.018329,0.018467,-0.545256,0.002513,0.952882
1105,534.303,0.551857,0.265763,0.164998,-0.010355,0.023869,-0.560007,-0.001668,0.284559
1106,537.563,0.544176,0.271015,0.164997,0.030599,-0.012734,-0.510231,-0.002801,-0.931129


In [29]:
final_pos

,time,pos_x,pos_y,pos_z,theta,vel_x,vel_y,vel_z,theta_dot
0,7.132,0.542186,-0.360492,0.164997,-0.009781,4.954050e-07,-2.541140e-04,-0.000059,3.948910e-04
1,10.392,0.545381,-0.340595,0.164997,-0.013830,1.763940e-06,-3.681590e-03,-0.000335,5.481570e-04
2,13.652,0.539915,-0.330229,0.164994,-0.022218,7.928710e-05,-2.356040e-03,-0.005591,2.299700e-08
3,16.902,0.542082,-0.320507,0.164996,0.013254,1.247920e-06,-1.499540e-03,0.001249,-7.674700e-04
4,20.172,0.547081,-0.304548,0.164997,-0.002957,7.770530e-04,9.970200e-07,0.001435,1.606120e-04
...,...,...,...,...,...,...,...,...,...
1103,529.383,0.540656,-0.284653,0.164995,0.023765,-9.328240e-05,-2.568940e-03,-0.005300,-2.908390e-08
1104,532.643,0.549084,-0.289120,0.165000,0.035046,-2.438680e-03,-8.672910e-05,0.007042,3.454040e-09
1105,535.903,0.548595,-0.286059,0.165000,-0.026814,-3.254900e-03,1.940180e-04,0.005992,1.814400e-07
1106,539.173,0.541819,-0.281863,0.165000,-0.030011,1.118310e-04,-2.451000e-03,0.004339,3.349360e-08


In [30]:
# tensors
x0_ = torch.tensor(initial_pos.pos_y)
xf_ = torch.tensor(final_pos.pos_y)
dt_ = torch.tensor(final_pos.time - initial_pos.time )
v0_ = torch.tensor(initial_pos.vel_y)
mu_ = torch.full(x0_.size(), 0.1, requires_grad=True)

In [31]:
# gravity constant
g = 9.81
#loss
loss = nn.MSELoss()
cost = 1e+8
#optimizer
optimizer = optim.SGD([mu_], lr=0.1, momentum=0.9)

In [32]:
while(cost)>1e-8:
    optimizer.zero_grad()
    xf_est_ = x0_+v0_*dt_+0.5*mu_*g*dt_**2
    cost = loss(xf_est_,xf_)
    cost.backward()
    optimizer.step()

In [33]:
mu_.mean()

tensor(0.0287, grad_fn=<MeanBackward0>)